# Thư viện

In [40]:
from lightgbm import LGBMRegressor
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sklearn
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

from sklearn.svm import LinearSVR

In [2]:
# from tensorflow import keras

from keras.layers import Input,Dense
from keras.models import Model
from keras.optimizers import Nadam
from keras.callbacks import ModelCheckpoint

Using TensorFlow backend.
C:\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (type, 1) or '1type

# Load dataset

In [3]:
rail_data = pd.read_csv('thegurus-opendata-renfe-trips.csv')

In [4]:
rail_data.head(10)

,id,company,origin,destination,departure,arrival,duration,vehicle_type,vehicle_class,price,fare,seats,meta,insert_date
0,1,renfe,MADRID,BARCELONA,2019-04-18 05:50:00,2019-04-18 08:55:00,3.08,AVE,Preferente,68.95,Promo,NaN,{},2019-04-11 21:49:46
1,2,renfe,MADRID,BARCELONA,2019-04-18 13:25:00,2019-04-18 16:24:00,2.98,AVE-TGV,Turista,107.70,Flexible,NaN,{},2019-04-11 21:49:46
2,3,renfe,MADRID,BARCELONA,2019-04-18 06:30:00,2019-04-18 09:20:00,2.83,AVE,Turista,75.40,Promo,NaN,{},2019-04-11 21:49:46
3,4,renfe,MADRID,BARCELONA,2019-04-18 15:30:00,2019-04-18 18:40:00,3.17,AVE,Preferente,NaN,Promo,NaN,{},2019-04-11 21:49:46
4,5,renfe,MADRID,BARCELONA,2019-04-18 07:00:00,2019-04-18 09:30:00,2.50,AVE,Turista Plus,106.75,Promo,NaN,{},2019-04-11 21:49:46
5,6,renfe,MADRID,BARCELONA,2019-04-18 06:30:00,2019-04-18 09:20:00,2.83,AVE,Turista,75.40,Promo,NaN,{},2019-04-11 21:49:46
6,7,renfe,MADRID,BARCELONA,2019-04-18 07:30:00,2019-04-18 10:40:00,3.17,AVE,Turista Plus,90.50,Promo,NaN,{},2019-04-11 21:49:46
7,8,renfe,MADRID,BARCELONA,2019-04-18 19:00:00,2019-04-18 21:30:00,2.50,AVE,Preferente,115.65,Promo,NaN,{},2019-04-11 21:49:46
8,9,renfe,MADRID,BARCELONA,2019-04-18 08:00:00,2019-04-18 10:30:00,2.50,AVE,Turista,88.95,Promo,NaN,{},2019-04-11 21:49:46
9,10,renfe,MADRID,BARCELONA,2019-04-18 08:00:00,2019-04-18 10:30:00,2.50,AVE,Turista,88.95,Promo,NaN,{},2019-04-11 21:49:46


* **insert_date** : ngày mà dòng dữ liệu được đưa vào data => ngày đặt vé
* **origin /destination** : nơi khởi hành/ nơi đến
* **departure / arrival** : thời gian khởi hành/ thời gian đến 
* **duration** : thời lượng di chuyển (tính bằng h) 
* **fare** : loại vé (khuyến mãi, ưu đãi thương mại,...)
* **price** : euros
* **vehicle_class** : hạng vé (ticket class, tourist, business, etc.)
* **seats** : số lượng chỗ ngồi

In [5]:
print('Shape of rail data: ', rail_data.shape)

Shape of rail data:  (38753060, 14)


In [6]:
rail_data.describe()

,id,duration,price,seats
count,3.875306e+07,3.875306e+07,2.959311e+07,1.247421e+06
mean,1.937653e+07,2.778496e+00,5.822162e+01,2.210353e+02
std,1.118704e+07,1.486362e+00,2.476530e+01,7.232385e+01
min,1.000000e+00,3.700000e-01,0.000000e+00,6.000000e+00
25%,9.688266e+06,1.800000e+00,3.920000e+01,2.050000e+02
50%,1.937653e+07,2.530000e+00,5.570000e+01,2.390000e+02
75%,2.906480e+07,2.980000e+00,7.540000e+01,2.620000e+02
max,3.875306e+07,1.242000e+01,3.428000e+02,3.340000e+02


In [7]:
rail_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38753060 entries, 0 to 38753059
Data columns (total 14 columns):
 #   Column         Dtype  
---  ------         -----  
 0   id             int64  
 1   company        object 
 2   origin         object 
 3   destination    object 
 4   departure      object 
 5   arrival        object 
 6   duration       float64
 7   vehicle_type   object 
 8   vehicle_class  object 
 9   price          float64
 10  fare           object 
 11  seats          float64
 12  meta           object 
 13  insert_date    object 
dtypes: float64(3), int64(1), object(10)
memory usage: 4.0+ GB


+ Set **insert_date**, **departure** và **arrival** bằng đơn vị thời gian: datetime64.

In [8]:
for item in ['insert_date', 'departure', 'arrival']:
    rail_data[item] = pd.to_datetime(rail_data[item])

## Set departure và arrival thành các đơn vị thời gian riêng

In [9]:
for item in ['insert_date', 'departure', 'arrival']:
    date_col = rail_data[item]
    rail_data[item + '_hour'] = date_col.dt.hour
    rail_data[item + '_minute'] = date_col.dt.minute
    rail_data[item + '_day'] = date_col.dt.day
    rail_data[item + '_month'] = date_col.dt.month
    rail_data[item + '_weekday'] = date_col.dt.day_name()
    
    del rail_data[item]

# Tiền xử lý
## Điền các giá trị thiếu

In [10]:
# Quan sát giá trị thiếu
rail_data.isnull().sum()

id                            0
company                       0
origin                        0
destination                   0
duration                      0
vehicle_type                  0
vehicle_class           8034239
price                   9159949
fare                    8034239
seats                  37505639
meta                          0
insert_date_hour              0
insert_date_minute            0
insert_date_day               0
insert_date_month             0
insert_date_weekday           0
departure_hour                0
departure_minute              0
departure_day                 0
departure_month               0
departure_weekday             0
arrival_hour                  0
arrival_minute                0
arrival_day                   0
arrival_month                 0
arrival_weekday               0
dtype: int64

In [11]:
# Cột seats thiếu quá nhiều dữ liệu, 37tr5 giá trị thiếu trên 38tr7 dòng => xóa cột seats
rail_data.drop(columns=['seats'], inplace=True)

In [12]:
# Điền hai cột vehicle_class và fare bằng mode
for i in ['vehicle_class', 'fare']: 
    rail_data[i].fillna(rail_data[i].mode()[0], inplace=True)

In [13]:
# Điền các giá trị price dựa trên thông tin của cột fare
rail_data.loc[rail_data.price.isnull(), 'price'] = rail_data.groupby('fare')['price'].transform('mean')

## Quan sát các cột chỉ có một giá trị

In [14]:
# Quan sát giá trị của các cột 
for item in rail_data.columns:
    print(item, ":", rail_data[item].unique())
    print('_'*30)

id : [       1        2        3 ... 38753058 38753059 38753060]
______________________________
company : ['renfe']
______________________________
origin : ['MADRID' 'SEVILLA' 'BARCELONA' 'VALENCIA' 'PONFERRADA' 'GRANADA' 'MALAGA'
 'CORDOBA' 'ZARAGOZA' 'LEON' 'ZAMORA' 'VALLADOLID' 'TOLEDO' 'SEGOVIA'
 'TARRAGONA' 'CUENCA' 'PALENCIA' 'CASTELLON' 'ALICANTE' 'CIUDAD REAL'
 'ALBACETE' 'CADIZ' 'LLEIDA' 'HUESCA' 'GUADALAJARA' 'GIRONA' 'CASTELLO'
 'LEÓN' 'MÁLAGA' 'CÓRDOBA']
______________________________
destination : ['BARCELONA' 'SEVILLA' 'VALENCIA' 'MADRID' 'PONFERRADA' 'GRANADA'
 'ZARAGOZA' 'MALAGA' 'CORDOBA' 'LEON' 'TOLEDO' 'GUADALAJARA' 'VALLADOLID'
 'ZAMORA' 'CIUDAD REAL' 'CUENCA' 'SEGOVIA' 'TARRAGONA' 'CASTELLON'
 'LLEIDA' 'ALICANTE' 'ALBACETE' 'HUESCA' 'CADIZ' 'GIRONA' 'PALENCIA'
 'CASTELLO' 'CÓRDOBA' 'LEÓN' 'MÁLAGA']
______________________________
duration : [ 3.08  2.98  2.83  3.17  2.5   2.75  3.07  3.    2.85  3.15  9.37  2.92
  2.53  2.7   2.73  2.63  2.58  2.6   2.67  7.68  2.55

In [15]:
# Quan sát giá trị của các cột 
for item in rail_data.columns:
    print(item, ":", rail_data[item].unique().shape[0])
    print('_'*30)

id : 38753060
______________________________
company : 1
______________________________
origin : 30
______________________________
destination : 30
______________________________
duration : 388
______________________________
vehicle_type : 30
______________________________
vehicle_class : 11
______________________________
price : 1503
______________________________
fare : 12
______________________________
meta : 1059814
______________________________
insert_date_hour : 24
______________________________
insert_date_minute : 60
______________________________
insert_date_day : 31
______________________________
insert_date_month : 11
______________________________
insert_date_weekday : 7
______________________________
departure_hour : 21
______________________________
departure_minute : 60
______________________________
departure_day : 31
______________________________
departure_month : 12
______________________________
departure_weekday : 7
______________________________
arrival_hour : 22

In [16]:
# Drop cột company
rail_data.drop(columns=['company'], inplace=True)

# Drop cột id
rail_data.drop(columns=['id'], inplace=True)

# Drop cột meta
rail_data.drop(columns=['meta'], inplace=True)

## Quan sát tương quan

In [17]:
corr = rail_data.corr()
corr.style.background_gradient(cmap='coolwarm')

,duration,price,insert_date_hour,insert_date_minute,insert_date_day,insert_date_month,departure_hour,departure_minute,departure_day,departure_month,arrival_hour,arrival_minute,arrival_day,arrival_month
duration,1.000000,-0.060425,0.013611,0.015099,-0.011389,0.067451,-0.164818,0.003409,-0.007080,0.039851,0.091926,0.006675,-0.007017,0.040012
price,-0.060425,1.000000,0.002901,0.042345,-0.000217,0.026407,0.024687,-0.159672,0.004302,-0.014398,0.020914,-0.023671,0.004060,-0.014441
insert_date_hour,0.013611,0.002901,1.000000,-0.013294,0.009946,0.002947,-0.001647,-0.004565,-0.002226,0.004964,0.001171,0.002150,-0.002208,0.004965
insert_date_minute,0.015099,0.042345,-0.013294,1.000000,0.109777,-0.066569,-0.000982,0.004174,0.013164,-0.052533,0.019032,0.005680,0.013219,-0.052592
insert_date_day,-0.011389,-0.000217,0.009946,0.109777,1.000000,-0.234496,0.004970,-0.001913,-0.008409,-0.107808,-0.001487,0.003298,-0.008497,-0.107791
insert_date_month,0.067451,0.026407,0.002947,-0.066569,-0.234496,1.000000,-0.012310,-0.030483,-0.025130,0.931431,0.019122,-0.021474,-0.025046,0.931404
departure_hour,-0.164818,0.024687,-0.001647,-0.000982,0.004970,-0.012310,1.000000,-0.058907,0.001915,-0.011099,0.752432,-0.012765,0.001727,-0.010616
departure_minute,0.003409,-0.159672,-0.004565,0.004174,-0.001913,-0.030483,-0.058907,1.000000,-0.002041,-0.020633,0.019031,-0.117658,-0.001873,-0.020683
departure_day,-0.007080,0.004302,-0.002226,0.013164,-0.008409,-0.025130,0.001915,-0.002041,1.000000,-0.146198,-0.000872,0.002994,0.996466,-0.145727
departure_month,0.039851,-0.014398,0.004964,-0.052533,-0.107808,0.931431,-0.011099,-0.020633,-0.146198,1.000000,0.011853,-0.026020,-0.145763,0.999931


=> departure_day, departure_month và arrival_day, arrival_month có correlation rất lớn với nhau. Vậy có thể bỏ 1 trong hai đi.

In [18]:
# Bỏ cột arrival_day, arrival_month và arrival_weekday
rail_data.drop(columns=['arrival_day', 'arrival_month', 'arrival_weekday'], inplace=True)

# Bỏ cột insert_date_hour, insert_date_minute, insert_date_day, insert_date_month, departure_hour
rail_data.drop(columns=['insert_date_hour', 'insert_date_minute', 'insert_date_day', 'insert_date_month'], inplace=True)

## Phân tích về giá vé
Giá vé tuy là một số, nhưng cũng là chi phí đại diện một tập các chuyến đi nhất định 

In [19]:
price_cnts = rail_data['price'].value_counts()
price_cnts.head(20)

57.337979     8667274
28.350000      967550
63.400000      823254
85.100000      771258
76.300000      722002
55.700000      594450
107.700000     561260
31.100000      524666
60.300000      520342
47.300000      503027
53.400000      485400
75.400000      477241
39.450000      465747
72.200539      421937
37.800000      410991
33.650000      377279
108.900000     373643
66.750000      368746
81.200000      331162
100.400000     322210
Name: price, dtype: int64

In [20]:
price_cnts.tail(20)

109.80    1
93.59     1
48.62     1
30.55     1
46.81     1
109.12    1
102.80    1
149.94    1
81.90     1
90.98     1
21.80     1
21.10     1
68.54     1
35.04     1
36.55     1
169.22    1
137.65    1
49.79     1
45.35     1
60.57     1
Name: price, dtype: int64

## Chia dữ liệu và nhãn cho mô hình dự đoán giá vé

In [36]:
X_df = rail_data.drop(columns = 'price') 
y = rail_data['price'].values

In [37]:
# Dùng One hot encoder để xử lý các thông tin danh nghĩa
encoder = OneHotEncoder()
X = encoder.fit_transform(X_df.values)
X

<38753060x721 sparse matrix of type '<class 'numpy.float64'>'
	with 542542840 stored elements in Compressed Sparse Row format>

# Model
## Chia train test 

In [24]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.01, random_state= 42
    )

print(X_train.shape)
print(X_test.shape)

print(y_train.shape)
print(y_test.shape)

(38365529, 721)
(387531, 721)
(38365529,)
(387531,)


## Hàm đánh giá

In [25]:
def MSE(model, X, y_true, name):
    y_pred = model.predict(X)
    mse = mean_squared_error(y_true, y_pred)
    print('Mean Squared Error for {}: {}'.format(name, mse))

def evaluation(model, X_train, y_train, X_test, y_test):
    train_score = model.score(X_train, y_train)
    test_score = model.score(X_test, y_test)

    print("Train Score:", train_score)
    print("Test Score:", test_score)
    print()
        
    MSE(model, X_train, y_train, 'training set')
    MSE(model, X_test, y_test, 'test set')

## LinearSVR model

In [26]:
%%time
svm = LinearSVR()
svm.fit(X_train, y_train)

Wall time: 15min 40s


LinearSVR()

In [27]:
evaluation(svm, X_train, y_train, X_test, y_test)

Train Score: 0.7029044738115887
Test Score: 0.7037699956103582

Mean Squared Error for training set: 140.11312148552835
Mean Squared Error for test set: 139.99467106082753


## Gradient boosting

In [46]:
%%time
gbr = LGBMRegressor(n_estimators=5000)
gbr.fit(X_train, y_train)

Wall time: 27min 37s


LGBMRegressor(n_estimators=5000)

In [ ]:
evaluation(gbr, X_train, y_train, X_test, y_test)

Train Score: 0.9221191095147949
Test Score: 0.9217329727158199

